In [2]:
%pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from PyPDF2 import PdfReader
import boto3
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import BedrockEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# --- Embeddings Bedrock ---
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    client=bedrock_runtime
)

/tmp/ipykernel_1314/3531557948.py:3: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  bedrock_embeddings = BedrockEmbeddings(


In [5]:

# --- Embeddings Bedrock ---
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    client=bedrock_runtime
)

In [6]:
# --- Charger le PDF ---
s3 = boto3.client("s3")
bucket = "sagemaker-us-east-1-745039059599"
key = "data_juridique/codes/LEGITEXT000006072050.pdf"
local_path = "/tmp/LEGITEXT000006072050.pdf"
s3.download_file(bucket, key, local_path)

In [7]:
reader = PdfReader(local_path)
text = ""
for i in range(5, min(200, len(reader.pages))):
    text += reader.pages[i].extract_text() or ""

In [8]:
documents = [Document(page_content=text, metadata={"source": key})]

In [9]:
# --- Split en chunks ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(documents)

In [11]:
# --- Créer et sauvegarder FAISS ---
vectorstore_faiss = FAISS.from_documents(split_docs, bedrock_embeddings)


In [12]:
local_faiss_folder = "/tmp/faiss_index"
vectorstore_faiss.save_local(local_faiss_folder)

In [13]:


# --- Vérification ---
print("Vector store FAISS sauvegardé dans :", local_faiss_folder)
print("Fichiers :", os.listdir(local_faiss_folder))






Vector store FAISS sauvegardé dans : /tmp/faiss_index
Fichiers : ['index.faiss', 'index.pkl']


Save to S3


In [14]:
import os
import boto3

# Configuration S3
bucket = "sagemaker-us-east-1-745039059599"  
s3_prefix = "faiss_index/"       # Dossier dans S3 où les fichiers seront stockés
local_folder = "/tmp/faiss_index"

In [15]:
# Créer le client S3
s3 = boto3.client("s3")

In [17]:

# Upload de tous les fichiers du dossier local vers S3
for filename in os.listdir(local_folder):
    local_file_path = os.path.join(local_folder, filename)
    s3_key = os.path.join(s3_prefix, filename)
    s3.upload_file(local_file_path, bucket, s3_key)
    print(f"Fichier uploadé : {s3_key}")

print("✅ Tous les fichiers FAISS ont été uploadés sur S3.")


Fichier uploadé : faiss_index/index.faiss
Fichier uploadé : faiss_index/index.pkl
✅ Tous les fichiers FAISS ont été uploadés sur S3.
